<hr/>

# Loan Repayment Prediction
[***By Jagadeesh Gajula***](https://www.kaggle.com/ztrimus) <br>
**March 2 2021**
<hr/>

Given historical data on loans given out with information on whether or not the borrower defaulted (charge-off), can we build a model that can predict wether or nor a borrower will pay back their loan? This way in the future when **we get a new potential customer we can assess whether or not they are likely to pay back the loan.**

## Table of Contents

* [1. Setup and Import](#1)
* [2. Limit the Feature Space](#2)
 * [2.1 Drop features missing more than 30% data](#2.1)
 * [2.2 Only keep loan features known to potential investors](#2.2)
* [3. Data Analysis and Pre-processing](#3)
 * [3.1 Overlook on Data](#3.1)
 * [3.2  Pre-processing on each Variable](#3.2)
* [4. More Pre-processing](#4)
 * [4.1 Convert loan status to 0/1 charge-off indicator](#4.1)
 * [4.2 Create dummy variables](#4.2)
 * [4.3 Train/test split](#4.3)
* [5. Linear Dependence of Charge-off on the Predictors](#5)
 * [5.1 Pearson correlation](#5.1)
* [6. Model Training and Testing](#6)
 * [6.1 Neural Network](#6.1)
 * [6.2 Logistic regression with SGD training](#6.2)
 * [6.3 Random forest classifier](#6.3)
 * [6.4 Tune hyperparameters on the chosen model more finely](#6.4)
 * [6.5 Test set evaluation](#6.5)
* [7. Conclusion](#7)

<a id="1"></a>
# 1. Setup and Import
<hr>

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import scipy as sp

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.style.use('ggplot')
sns.set(style='whitegrid')

# scaling and train test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# pandas options
pd.set_option('display.max_colwidth', 1000, 'display.max_rows', None, 'display.max_columns', None)
pd.set_option("display.precision", 4) # Show numbers only with 2 decimal places

# machine learning algorithm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

# creating a model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

# evaluation on test data
from sklearn.metrics import classification_report,confusion_matrix


In [ ]:
%%time
# Create dataframe from dataset file
loans = pd.read_csv('accepted_2007_to_2018Q4.csv',low_memory=False)
loans.head()

<a id="2"></a>
# 2. Limit the Feature Space

The full dataset has 150 features for each loan. We'll select features in two steps:

1. Drop features with more than 30% of their data missing.
2. Of the remaining features, choose only those that would be available to an investor before deciding to fund the loan.

<a id="2.1"></a>
## 2.1 Drop features missing more than 30% data

In [ ]:
missing_fractions = loans.isnull().mean().sort_values(ascending=False)
missing_fractions

In [ ]:
plt.figure(figsize=(6,3), dpi=90)
missing_fractions.plot.hist()
plt.title('Histogram of Feature Incompleteness')
plt.xlabel('Fraction of data missing')
plt.ylabel('Feature count')

From the above histogram, we see there's a large gap between features missing "some" data (&lt;20%) and those missing "lots" of data (&gt;40%). Because it's generally very difficult to accurately impute data with more than 30% missing values, we drop such columns. First store all variables missing more than 30% data in an alphabetical list:

In [ ]:
drop_list = sorted(list(missing_fractions[missing_fractions > 0.4].index))
print(drop_list)
print("\n\n Drop Features: ", len(drop_list))

In [ ]:
# Drop these features
loans.drop(labels=drop_list, axis=1, inplace=True)

In [ ]:
loans.shape

<a id="2.2"></a>
## 2.2 Only keep loan features known to potential investors

In [ ]:
loans.columns.tolist()

For each of these features, we check the description in the Data Dictionary and only keep the features that would have been available to investors considering an investment in the loan. These include features in the loan application, and any features added by LendingClub when the loan listing was accepted, such as the loan grade and interest rate.

I'm using my best available knowledge to determine which loan features are known to potential investors. I am not an investor on LendingClub, so my knowledge of the LendingClub investment process is not exact. When in doubt, I err on the side of dropping the feature.

In [ ]:
keep_list = ['addr_state', 'annual_inc', 'application_type', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'grade', 'home_ownership', 'id', 'initial_list_status', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'loan_status', 'mort_acc', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'term', 'title', 'total_acc', 'verification_status', 'zip_code']
print("Keep features: ", len(keep_list))

In [ ]:
keep_list

In [ ]:
keep_str = """acc_open_past_24mths
addr_state
annual_inc
application_type
avg_cur_bal
chargeoff_within_12_mths
dti
earliest_cr_line
emp_length
emp_title
fico_range_high
fico_range_low
grade
home_ownership
initial_list_status
installment
int_rate
issue_d
loan_amnt
mort_acc
open_acc
pub_rec
pub_rec_bankruptcies
purpose
revol_bal
revol_util
sub_grade
term
title
total_acc
zip_code"""

keep_str = keep_str.split()

In [ ]:
drop_list = [col for col in loans.columns if col not in keep_list]
print(drop_list)

print("\n\nNumber of features needed to be dropped: ",len(drop_list))

In [ ]:
loans.drop(labels=drop_list, axis=1, inplace=True)
loans.shape

<a id="3"></a>
# 3. Data Analysis and Pre-processing

* Overlook on Data
* Pre-processing on each Variable

<a id="3.1"></a>
## 3.1 Overlook on Data

In [ ]:
print("Records: ", loans.shape[0], "\nFeatures: ", loans.shape[1])
print("\nInformation of Dataset\n-------")
loans.info()

In [ ]:
loans.describe(include=np.object)

In [ ]:
# Target Variable
loans['loan_status'].value_counts(dropna=False)

We're going to try to learn differences in the features between completed loans that have been fully paid or charged off. We won't consider loans that are current, don't meet the credit policy, defaulted, or have a missing status. So we only keep the loans with status "Fully Paid" or "Charged Off."

In [ ]:
loans = loans.loc[loans['loan_status'].isin(['Fully Paid', 'Charged Off'])]

In [ ]:
# Categories in percentage
loans['loan_status'].value_counts()

* This is an imbalance problem, because we have a lot more entries of people that fully paid their loans then people that did not pay back.
* We can expect to probably do very well in terms of accuracy but our precision and recall are going to be the true metrics that we will have to evaluate our model based off of.

* In the loan amount distribution we can see spikes in even ten thousend dollar, so this is indicating that there are certain amounts that are basically standard loans.

<a id="3.2"></a>
## 3.2 Pre-processing on each Variable

We'll inspect each feature individually, and do the following:

1. Drop the feature if it is not useful for predicting the loan status.
2. View summary statistics and visualize the data, plotting against the loan status.
3. Modify the feature to make it useful for modeling, if necessary.

We define a function for plotting a variable and comparing with the loan status:

In [ ]:
def plot_var(col_name, full_name, continuous):
    """
    Visualize a variable with and without faceting on the loan status.
    - col_name is the variable name in the dataframe
    - full_name is the full variable name
    - continuous is True if the variable is continuous, False otherwise
    """
    f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,3), dpi=90)
    
    # Plot without loan status
    if continuous:
        sns.distplot(loans.loc[loans[col_name].notnull(), col_name], kde=False, ax=ax1)
    else:
        sns.countplot(loans[col_name], order=sorted(loans[col_name].unique()), color='#5975A4', saturation=1, ax=ax1)
    ax1.set_xlabel(full_name)
    ax1.set_ylabel('Count')
    ax1.set_title(full_name)

    # Plot with loan status
    if continuous:
        sns.boxplot(x=col_name, y='loan_status', data=loans, ax=ax2)
        ax2.set_ylabel('')
        ax2.set_title(full_name + ' by Loan Status')
    else:
        charge_off_rates = loans.groupby(col_name)['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']
        sns.barplot(x=charge_off_rates.index, y=charge_off_rates.values, color='#5975A4', saturation=1, ax=ax2)
        ax2.set_ylabel('Fraction of Loans Charged-off')
        ax2.set_title('Charge-off Rate by ' + full_name)
    ax2.set_xlabel(full_name)
    
    plt.tight_layout()

Print the remaining features for future reference:

In [ ]:
print(list(loans.columns))

### 3.2.1 id

Data Dictionary: "A unique [LendingClub] assigned ID for the loan listing."

Are all the IDs unique?

In [ ]:
loans['id'].describe()

Yes, they are all unique. The ID is not useful for modeling, either as a categorical variable (there are too many distinct values) or as a numerical variable (the IDs vary wildly in magnitude, likely without any significance), so we drop this variable.

In [ ]:
loans.drop('id', axis=1, inplace=True)

### 3.2.2 loan_amnt

Data Dictionary: "The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."

In [ ]:
loans['loan_amnt'].describe()

Loan amounts range from \$500 to \$40,000, with a median of \$12,000.

In [ ]:
plot_var('loan_amnt', 'Loan Amount', continuous=True)

Charged-off loans tend to have higher loan amounts. Let's compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['loan_amnt'].describe()

### 3.2.3 term

Data Dictionary: "The number of payments on the loan. Values are in months and can be either 36 or 60."

In [ ]:
loans['term'].value_counts(dropna=False)

Convert `term` to integers.

In [ ]:
loans['term'] = loans['term'].apply(lambda s: np.int8(s.split()[0]))
loans['term'].value_counts(normalize=True)

Compare the charge-off rate by loan period:

In [ ]:
loans.groupby('term')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

About 75% of the completed loans have three-year periods, and the rest have five-year periods. Loans with five-year periods are more than twice as likely to charge-off as loans with three-year periods.

### 3.2.4 int_rate

Data Dictionary: "Interest Rate on the loan."

In [ ]:
# Let's remove % percent sign from `int_rate` and convert it into `float`
loans.int_rate = loans.int_rate.astype(str).map(lambda x: x.replace('%','')).astype(np.float64)

In [ ]:
loans['int_rate'].describe()

Interest rates range from 5.32% to 31% (!) with a median of 13.2%.

In [ ]:
plot_var('int_rate', 'Interest Rate', continuous=True)

Charged-off loans tend to have much higher interest rates. Let's compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['int_rate'].describe()

### 3.2.5 installment
<a id="3.2.5"></a>

Data Dictionary: "The monthly payment owed by the borrower if the loan originates."

In [ ]:
loans['installment'].describe()

Installments range from \$4.93 to \$1,714, with a median of \$377.

In [ ]:
plot_var('installment', 'Installment', continuous=True)

Charged-off loans tend to have higher installments. Let's compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['installment'].describe()

Loans that charge off have \$30 higher installments on average.

### 3.2.6 grade, sub_grade

Data Dictionary for `grade`: "LendingClub assigned loan grade."

Data Dictionary for `sub_grade`: "LendingClub assigned loan subgrade."

What are the possible values of `grade` and `sub_grade`?

In [ ]:
print(sorted(loans['grade'].unique()))

In [ ]:
print(sorted(loans['sub_grade'].unique()))

The grade is implied by the subgrade, so let's drop the grade column.

In [ ]:
loans.drop('grade', axis=1, inplace=True)

In [ ]:
plot_var('sub_grade', 'Subgrade', continuous=False)

There's a clear trend of higher probability of charge-off as the subgrade worsens.

### 3.2.7 emp_title

Data Dictionary: "The job title supplied by the Borrower when applying for the loan."

In [ ]:
loans['emp_title'].describe()

There are too many different job titles for this feature to be useful, so we drop it.

In [ ]:
loans.drop(labels='emp_title', axis=1, inplace=True)

### 3.2.8 emp_length

Data Dictionary: "Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years." The actual data does not match this description:

In [ ]:
loans['emp_length'].value_counts(dropna=False).sort_index()

Note there are 117126 loans without data on the length of employment.

Convert `emp_length` to integers:

In [ ]:
loans['emp_length'].replace(to_replace='10+ years', value='10 years', inplace=True)

In [ ]:
loans['emp_length'].replace('< 1 year', '0 years', inplace=True)

In [ ]:
def emp_length_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

In [ ]:
loans['emp_length'] = loans['emp_length'].apply(emp_length_to_int)

In [ ]:
loans['emp_length'].value_counts(dropna=False).sort_index()

In [ ]:
plot_var('emp_length', 'Employment Length', continuous=False)

In [ ]:
loans.drop('emp_length', axis=1, inplace=True)

Loan status does not appear to vary much with employment length on average, except for a small drop in charge-offs for borrowers with over 10 years of employment.

### 3.2.9 home_ownership

Data Dictionary: "The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER."

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

Replace the values `ANY` and `NONE` with `OTHER`:

In [ ]:
loans['home_ownership'].replace(['NONE', 'ANY'], 'OTHER', inplace=True)

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

In [ ]:
plot_var('home_ownership', 'Home Ownership', continuous=False)

There appear to be large differences in charge-off rates by home ownership status. Renters and homeowners have a higher probability of charge-off. Let's compare the charge-off rates:

In [ ]:
loans.groupby('home_ownership')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

### 3.2.10 annual_inc

Data Dictionary: "The self-reported annual income provided by the borrower during registration."

In [ ]:
loans['annual_inc'].describe()

Annual income ranges from \$0 to \$9,550,000, with a median of \$65,000. Because of the large range of incomes, we should take a log transform of the annual income variable.

In [ ]:
loans['log_annual_inc'] = loans['annual_inc'].apply(lambda x: np.log10(x+1))

In [ ]:
loans.drop('annual_inc', axis=1, inplace=True)

In [ ]:
loans['log_annual_inc'].describe()

In [ ]:
plot_var('log_annual_inc', 'Log Annual Income', continuous=True)

It appears that individuals with higher income are more likely to pay off their loans. Let's compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['log_annual_inc'].describe()

### 3.2.11 verification_status

Data Dictionary: "Indicates if income was verified by [Lending Club], not verified, or if the income source was verified."

In [ ]:
plot_var('verification_status', 'Verification Status', continuous=False)

### 3.2.12 issue_d

Data Dictionary: "The month which the loan was funded."

Because we're only using variables available to investors before the loan was funded, `issue_d` will not be included in the final model. We're keeping it for now just to perform the train/test split later, then we'll drop it.

### 3.2.13 purpose

Data Dictionary: "A category provided by the borrower for the loan request."

In [ ]:
loans['purpose'].value_counts()

Calculate the charge-off rates by purpose:

In [ ]:
loans.groupby('purpose')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off'].sort_values()

Notice that only 12% of completed loans for weddings have charged-off, but 30% of completed small business loans have charged-off.

### 3.2.14 title

Data Dictionary: "The loan title provided by the borrower."

In [ ]:
loans['title'].describe()

View the top 10 loan titles, and their frequencies:

In [ ]:
loans['title'].value_counts().head(10)

There are 60,298 different titles in the dataset, and based on the top 10 titles, the `purpose` variable appears to already contain this information. So we drop the `title` variable.

In [ ]:
loans.drop('title', axis=1, inplace=True)

### 3.2.15 zip_code, addr_state

Data Dictionary for `zip_code`: "The first 3 numbers of the zip code provided by the borrower in the loan application."

Data Dictionary for `addr_state`: "The state provided by the borrower in the loan application."

In [ ]:
loans['zip_code'].sample(5)

In [ ]:
loans['zip_code'].nunique()

In [ ]:
loans['addr_state'].sample(5)

In [ ]:
loans['addr_state'].nunique()

There are a lot of different zip codes, so let's just keep the state column.

In [ ]:
loans.drop(labels='zip_code', axis=1, inplace=True)

Calculate the charge-off rates by address state:

In [ ]:
loans.groupby('addr_state')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off'].sort_values()

The charge-off rate ranges from 13.0% in Washington, DC to 27.6% in Mississippi.

### 3.2.16 dti

Data Dictionary: "A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income."

In [ ]:
loans['dti'].describe()

Note sure if the values of -1 and 999 make sense...

There are several outliers that mess up our default plots. Plot a histogram for `dti` less than 60:

In [ ]:
plt.figure(figsize=(8,3), dpi=90)
sns.distplot(loans.loc[loans['dti'].notnull() & (loans['dti']<60), 'dti'], kde=False)
plt.xlabel('Debt-to-income Ratio')
plt.ylabel('Count')
plt.title('Debt-to-income Ratio')

How many of the `dti` values are "outliers" (above 60)?

In [ ]:
(loans['dti']>=60).sum()

Very few. Compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['dti'].describe()

Completed loans that are charged off tend to have higher debt-to-income ratios.

### 3.2.17 earliest_cr_line

Data Dictionary: "The month the borrower's earliest reported credit line was opened."

In [ ]:
loans['earliest_cr_line'].sample(5)

In [ ]:
loans['earliest_cr_line'].isnull().any()

Let's just retain the year for simplicity:

In [ ]:
loans['earliest_cr_line'] = loans['earliest_cr_line'].apply(lambda s: int(s[-4:]))

In [ ]:
loans['earliest_cr_line'].describe()

In [ ]:
plot_var('earliest_cr_line', 'Year of Earliest Credit Line', continuous=True)

Borrowers who are charged-off tend to have shorter lines of credit.

### 3.2.18 fico_range_low, fico_range_high

Data Dictionary for `fico_range_low`: "The lower boundary range the borrower’s FICO at loan origination belongs to."

Data Dictionary for `fico_range_high`: "The upper boundary range the borrower’s FICO at loan origination belongs to."

In [ ]:
loans[['fico_range_low', 'fico_range_high']].describe()

Check the Pearson correlation between these values:

In [ ]:
loans[['fico_range_low','fico_range_high']].corr()

We only need to keep one of the FICO scores. We'll take the average of the two and call it `fico_score`:

In [ ]:
loans['fico_score'] = 0.5*loans['fico_range_low'] + 0.5*loans['fico_range_high']

In [ ]:
loans.drop(['fico_range_high', 'fico_range_low'], axis=1, inplace=True)

In [ ]:
plot_var('fico_score', 'FICO Score', continuous=True)

There is a noticeable difference in FICO scores between fully paid and charged-off loans. Compare the summary statistics:

In [ ]:
loans.groupby('loan_status')['fico_score'].describe()

Loans that charge off have a FICO score 10 points lower on average.

### 3.2.19 open_acc

Data Dictionary: "The number of open credit lines in the borrower's credit file."

In [ ]:
plt.figure(figsize=(10,3), dpi=90)
sns.countplot(loans['open_acc'], order=sorted(loans['open_acc'].unique()), color='#5975A4', saturation=1)
_, _ = plt.xticks(np.arange(0, 90, 5), np.arange(0, 90, 5))
plt.title('Number of Open Credit Lines')

Is there a difference in number of credit lines between fully paid loans and charged-off loans?

In [ ]:
loans.groupby('loan_status')['open_acc'].describe()

### 3.2.20 pub_rec

Data Dictionary: "Number of derogatory public records."

In [ ]:
loans['pub_rec'].value_counts().sort_index()

Is there a difference in average public records between fully paid loans and charged-off loans?

In [ ]:
loans.groupby('loan_status')['pub_rec'].describe()

### 3.2.21 revol_bal

Data Dictionary: "Total credit revolving balance."

In [ ]:
loans['revol_bal'].describe()

Do a log transform:

In [ ]:
loans['log_revol_bal'] = loans['revol_bal'].apply(lambda x: np.log10(x+1))

In [ ]:
loans.drop('revol_bal', axis=1, inplace=True)

In [ ]:
plot_var('log_revol_bal', 'Log Revolving Credit Balance', continuous=True)

In [ ]:
loans.groupby('loan_status')['log_revol_bal'].describe()

There isn't a large difference in the means.

### 3.2.22 revol_util

Data Dictionary: "Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit."

In [ ]:
# Let's remove % percent sign from `revol_util` and convert it into `float`
loans.revol_util = loans.revol_util.map(lambda x: str(x).replace('%','')).astype(np.float64)
loans['revol_util'].head()

In [ ]:
loans['revol_util'].describe()

In [ ]:
plot_var('revol_util', 'Revolving Line Utilization', continuous=True)

In [ ]:
loans.groupby('loan_status')['revol_util'].describe()

### 3.2.23 total_acc

Data Dictionary: "The total number of credit lines currently in the borrower's credit file."

In [ ]:
plt.figure(figsize=(12,3), dpi=90)
sns.countplot(loans['total_acc'], order=sorted(loans['total_acc'].unique()), color='#5975A4', saturation=1)
_, _ = plt.xticks(np.arange(0, 176, 10), np.arange(0, 176, 10))
plt.title('Total Number of Credit Lines')

In [ ]:
loans.groupby('loan_status')['total_acc'].describe()

No large differences here.

### 3.2.24 initial_list_status

Data Dictionary: "The initial listing status of the loan. Possible values are – W, F." I'm not sure what this means.

In [ ]:
plot_var('initial_list_status', 'Initial List Status', continuous=False)

### 3.2.25 application_type

Data Dictionary: "Indicates whether the loan is an individual application or a joint application with two co-borrowers."

In [ ]:
loans['application_type'].value_counts()

Let's just compare the charge-off rates by application type:

In [ ]:
loans.groupby('application_type')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

Joint loans are slightly less likely to be charged-off.

### 3.2.26 mort_acc

Data Dictionary: "Number of mortgage accounts."

In [ ]:
loans['mort_acc'].describe()

Not sure how someone can have 51 mortgage accounts...but apparently they do. Check the top 10 values:

In [ ]:
loans['mort_acc'].value_counts().head(10)

In [ ]:
total_acc_avg = loans.groupby('total_acc').mean()['mort_acc'].fillna(0)

def fill_mort_acc(total_acc,mort_acc):
    '''
    Accepts the total_acc and mort_acc values for the row.
    Checks if the mort_acc is NaN , if so, it returns the avg mort_acc value
    for the corresponding total_acc value for that row.
    
    total_acc_avg here should be a Series or dictionary containing the mapping of the
    groupby averages of mort_acc per total_acc values.
    '''
    if mort_acc>=0:
        return total_acc_avg[total_acc]
    else:
        return mort_acc

In [ ]:
loans['mort_acc'] = loans.apply(lambda x: fill_mort_acc(x['total_acc'], x['mort_acc']), axis=1)

Compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['mort_acc'].describe()

Individuals who pay off their loans are more likely to have several mortgage accounts.

### 3.2.27 pub_rec_bankruptcies

Data Dictionary: "Number of public record bankruptcies."

In [ ]:
loans['pub_rec_bankruptcies'].value_counts().sort_index()

In [ ]:
plot_var('pub_rec_bankruptcies', 'Public Record Bankruptcies', continuous=False)

<a id="4"></a>
# 4. More Pre-processing

<a id="4.1"></a>
## 4.1 Convert loan status to 0/1 charge-off indicator

Change the response variable `loan_status` to a 0/1 variable, where 0 indicates fully paid and 1 indicates charge-off:

In [ ]:
loans['charged_off'] = (loans['loan_status'] == 'Charged Off').apply(np.uint8)
loans.drop('loan_status', axis=1, inplace=True)

In [ ]:
loans = loans.dropna().reset_index(drop=True)

<a id="4.2"></a>
## 4.2 Create dummy variables

How many variables do we currently have?

In [ ]:
loans.shape

If any categorical variables have missing values, we'll need to create NaN dummy variables for those. So first check which variables have missing data:

In [ ]:
missing_fractions = loans.isnull().mean().sort_values(ascending=False) # Fraction of data missing for each variable

In [ ]:
print(missing_fractions[missing_fractions > 0]) # Print variables that are missing data

There are no categorical variables with missing values, and therefore we don't need any `NaN` dummy variables.

Create dummy variables for the categorical variables:

In [ ]:
print(loans.columns)

In [ ]:
loans = pd.get_dummies(loans, columns=['sub_grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state', 'initial_list_status', 'application_type'], drop_first=True)

How many variables are there now?

In [ ]:
loans.shape

Check our data with the new dummy variables:

In [ ]:
loans.sample(5)

<a id="4.3"></a>
## 4.3 Train/test split

In [ ]:
# We'll make our modeling problem more realistic by performing the train/test split based on the month that the loan was funded. That is, we'll use loans funded on earlier dates to predict whether future loans will charge-off. The variable `issue_d` includes the month and year that the loan was funded.
# loans['issue_d'].sample(5)

# Are there any missing values?
# loans['issue_d'].isnull().any()

# No. Let's convert the issue dates to datetime objects:
# loans['issue_d'] = pd.to_datetime(loans['issue_d'])
# loans['issue_d'].sample(5)

# The new datetime values are all on the first day of the month. Check the summary statistics of the issue dates:
# loans['issue_d'].describe()
# There are only 154 unique issue dates over the 10-year period because we only have month/year information. In this particular dataset, the first loans were issued in June 2007, and the most recent loans were issued in March 2020. The busiest month was March 2016 with 57,553 loans funded in that month. What is the distribution of loans funded in each year?

# plt.figure(figsize=(6,3), dpi=90)
# loans['issue_d'].dt.year.value_counts().sort_index().plot.bar(color='darkblue')
# plt.xlabel('Year')
# plt.ylabel('Number of Loans Funded')
# plt.title('Loans Funded per Year')

# We'll form the test set from the most recent 10% of the loans.

# loans_train = loans.loc[loans['issue_d'] <  loans['issue_d'].quantile(0.9)]
# loans_test =  loans.loc[loans['issue_d'] >= loans['issue_d'].quantile(0.9)]
# Refer [this video](https://www.youtube.com/watch?v=idXEk3MjC9M) if you didn't understood the quantile. Check that we properly partitioned the loans:

# What is the test size?
# loans_test.shape[0] / loans.shape[0]
# About 10.8%. The partition looks good, so we can delete the original `loans` dataframe:
# The training set includes loans from June 2007 to April 2018. The test set includes loans from May 2018 to March 2020. Now we need to delete the `issue_d` variable, because it was not available before the loan was funded.

# del loans

# loans_train.drop('issue_d', axis=1, inplace=True)
# loans_test.drop('issue_d', axis=1, inplace=True)

# del loans_train, loans_test

# y_train = loans_train['charged_off']
# y_test = loans_test['charged_off']

# X_train = loans_train.drop('charged_off', axis=1)
# X_test = loans_test.drop('charged_off', axis=1)

In [ ]:
loans.drop('issue_d', axis=1, inplace=True)

In [ ]:
# Features
X = loans.drop('charged_off',axis=1)

# Label
y = loans['charged_off']

In [ ]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
print("X_train.shape: ", X_train.shape)
print("X_test.shape:  ", X_test.shape)
print("y_train.shape: ", y_train.shape)
print("y_test.shape:  ", y_test.shape)

<a id="5"></a>
# 5. Linear Dependence of Charge-off on the Predictors

On the training set, we compute the [Pearson correlation](https://www.youtube.com/watch?v=6fUYt1alA1U), $F$-statistic, and $p$ value of each predictor with the response variable `charged_off`.

In [ ]:
linear_dep = pd.DataFrame()

<a id="5.1"></a>
## 5.1 Pearson correlation
We use the Pearson correlation coefficient to examine the strength and direction of the linear relationship between two continuous variables.

The correlation coefficient can range in value from −1 to +1. The larger the absolute value of the coefficient, the stronger the relationship between the variables. For the Pearson correlation, an absolute value of 1 indicates a perfect linear relationship. A correlation close to 0 indicates no linear relationship between the variables. 

The sign of the coefficient indicates the direction of the relationship. If both variables tend to increase or decrease together, the coefficient is positive, and the line that represents the correlation slopes upward. If one variable tends to increase as the other decreases, the coefficient is negative, and the line that represents the correlation slopes downward.

* We can see a strong correlation between loan_amnt and installment. (The monthly payment owed by the borrower if the loan originates)

In [ ]:
for col in X_train.columns:
    linear_dep.loc[col, 'pearson_corr'] = X_train[col].corr(y_train)
linear_dep['abs_pearson_corr'] = abs(linear_dep['pearson_corr'])

In [ ]:
linear_dep

$F$-statistics:

In [ ]:
from sklearn.feature_selection import f_classif
for col in X_train.columns:
    mask = X_train[col].notnull()
    (linear_dep.loc[col, 'F'], linear_dep.loc[col, 'p_value']) = f_classif(pd.DataFrame(X_train.loc[mask, col]), y_train.loc[mask])

Sort the results by the absolute value of the Pearson correlation:

In [ ]:
linear_dep.sort_values('abs_pearson_corr', ascending=False, inplace=True)
linear_dep.drop('abs_pearson_corr', axis=1, inplace=True)

Reset the index:

In [ ]:
linear_dep.reset_index(inplace=True)
linear_dep.rename(columns={'index':'variable'}, inplace=True)

View the results for the top 20 predictors most correlated with `charged_off`:

In [ ]:
linear_dep.head(30)

The variables most linearly correlated with `charged_off` are the interest rate, loan period (term), FICO score, debt-to-income ratio, number of mortgages, income, the loan grade, and the loan amount.

Now view the results for the 20 least correlated predictors:

In [ ]:
linear_dep.tail(20)

It looks like the borrower's state of residence, the revolving balance, and several of the loan purposes are irrelevant for predicting charge-off.

<a id="6"></a>
# 6. Model Training and Testing

We implement machine learning pipelines consisting of one or more of the following steps, depending on the particular model:
1. Mean imputation of missing values
2. Dimension reduction using linear discriminant analysis (LDA)
3. Data standardization: rescaling to zero mean and unit variance
4. The chosen model

We will evaluate and compare the following models using a cross-validated AUROC score on the training set:
1. Neural Network
2. Logistic regression with SGD training
3. Random forest

We'll perform some hyperparameter tuning for each model to choose the most promising model, then more carefully tune the hyperparameters of the best-performing model.

<a id="6.1"></a>
## 6.1 Neural Network

In [ ]:
scaler = MinMaxScaler()

# fit and transfrom
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# everything has been scaled between 1 and 0
print('Max: ',X_train.max())
print('Min: ', X_train.min())

In [ ]:
import tensorflow as tf

assert tf.test.is_gpu_available()
assert tf.test.is_built_with_cuda()

In [ ]:
model = Sequential()

# input layer
model.add(Dense(X_train.shape[1],activation='relu'))
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(16,activation='relu'))
model.add(Dropout(0.2))

# output layer
model.add(Dense(1, activation='sigmoid'))

# compile model
model.compile(optimizer="adam", loss='binary_crossentropy')

In [ ]:
# Early Stopping
early_stop = EarlyStopping(patience=50)

In [ ]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=25,
          verbose = 4,
          batch_size=4096,
          validation_data=(X_test, y_test))



In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
predictions = model.predict_classes(X_test)

print('Classification Report:')
print(classification_report(y_test, predictions))
print('\n')
print('Confusion Matirx:')
print(confusion_matrix(y_test, predictions))

In [ ]:
rnd.seed(101)
random_ind = rnd.randint(0,len(loans))

new_customer = loans.drop('charged_off',axis=1).iloc[random_ind]
new_customer

In [ ]:
# we need to reshape this to be in the same shape of the training data that the model was trained on
model.predict_classes(new_customer.values.reshape(1,X_train.shape[1]))

In [ ]:
# the prediction was right
loans.iloc[random_ind]['charged_off']

<a id="6.2"></a>
## 6.2 Logistic regression with SGD training

The `SGDClassifier` estimator in scikit-learn implements linear classifiers (SVM, logistic regression, and others) with stochastic gradient descent (SGD) training. A particular linear classifier is chosen through the `loss` hyperparameter. Because we want to predict the probability of charge-off, we choose logistic regression (a probabilistic classifier) by setting `loss = 'log'`.

The machine learning pipeline:

In [ ]:
pipeline_sgdlogreg = Pipeline([
    ('imputer', SimpleImputer(copy=False)), # Mean imputation by default
    ('scaler', StandardScaler(copy=False)),
    ('model', SGDClassifier(loss='log', max_iter=1000, tol=1e-3, random_state=1, warm_start=True))
])

A small grid of hyperparameters to search over:

In [ ]:
param_grid_sgdlogreg = {
    'model__alpha': [10**-5, 10**-2, 10**1],
    'model__penalty': ['l1', 'l2']
}

Create the search grid object:

In [ ]:
grid_sgdlogreg = GridSearchCV(estimator=pipeline_sgdlogreg, param_grid=param_grid_sgdlogreg, scoring='roc_auc', n_jobs=-1, pre_dispatch=1, cv=5, verbose=1, return_train_score=False)

Conduct the grid search and train the final model on the whole dataset:

In [ ]:
grid_sgdlogreg.fit(X_train, y_train)

Mean cross-validated AUROC score of the best model:

In [ ]:
grid_sgdlogreg.best_score_

Best hyperparameters:

In [ ]:
grid_sgdlogreg.best_params_

<a id="6.3"></a>
## 6.3 Random forest classifier

Next we train a random forest model. Note that data standardization is not necessary for a random forest.

In [ ]:
pipeline_rfc = Pipeline([
    ('imputer', SimpleImputer(copy=False)),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=1))
])

The random forest takes very long to train, so we don't test different hyperparameter choices. We'll still use `GridSearchCV` for the sake of consistency.

In [ ]:
param_grid_rfc = {
    'model__n_estimators': [50] # The number of randomized trees to build
}

The AUROC will always improve (with decreasing gains) as the number of estimators increases, but it's not necessarily worth the extra training time and model complexity.

In [ ]:
grid_rfc = GridSearchCV(estimator=pipeline_rfc, param_grid=param_grid_rfc, scoring='roc_auc', n_jobs=-1, pre_dispatch=1, cv=5, verbose=1, return_train_score=False)

In [ ]:
grid_rfc.fit(X_train, y_train)

Mean cross-validated AUROC score of the random forest:

In [ ]:
grid_rfc.best_score_

Not quite as good as logistic regression, at least according to this metric.

<a id="6.4"></a>
## 6.4 Tune hyperparameters on the chosen model more finely

The three models performed quite similarly according to the AUROC:

In [ ]:
print('Cross-validated AUROC scores')
print(grid_sgdlogreg.best_score_, '- Logistic regression')
print(grid_rfc.best_score_, '- Random forest')

Logistic regression squeaked out ahead, and coupled with the fact that `SGDClassifier` trains much faster than the other two models, we'll select logistic regression as our final model. Now we'll tune the hyperparameters more finely.

In [ ]:
param_grid_sgdlogreg = {
    'model__alpha': np.logspace(-4.5, 0.5, 11), # Fills in the gaps between 10^-5 and 10^1
    'model__penalty': ['l1', 'l2']
}

print(param_grid_sgdlogreg)

In [ ]:
grid_sgdlogreg = GridSearchCV(estimator=pipeline_sgdlogreg, param_grid=param_grid_sgdlogreg, scoring='roc_auc', n_jobs=-1, pre_dispatch=1, cv=5, verbose=1, return_train_score=False)

In [ ]:
grid_sgdlogreg.fit(X_train, y_train)

Mean cross-validated AUROC score of the best model:

In [ ]:
grid_sgdlogreg.best_score_

Best hyperparameters:

In [ ]:
grid_sgdlogreg.best_params_

By some coincidence, the optimal hyperparameters here are the same as from our first grid search for logistic regression!

<a id="6.5"></a>
## 6.5 Test set evaluation

Now we can finally see how our chosen model performs on the test data (the most recent 10% of the loans).

In [ ]:
y_score = grid_sgdlogreg.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_score)

The test set AUROC score is somewhat lower than the cross-validated score (0.713).

<a id="7"></a>
# 7. Conclusion

We applied machine learning methods to predict the probability that a requested loan on LendingClub will charge off. After training and evaluating three different models (logistic regression, random forest, and k-nearest neighbors), we found that all three performed similarly according to a cross-validated AUROC score on the training data. We selected logistic regression (with ridge penalty) because it was the fastest model to train, and this model obtained an AUROC score of 0.689 on a test set consisting of the most recent 10% of the loans.

This model, while far from perfect, can provide a somewhat informed prediction of the likelihood that a loan will charge off, using only data available to potential investors before the loan is fully funded.

We also found that, according to linear measures of correlation between the predictors and the response, the most important variables for predicting charge-off are the loan interest rate and term, and the borrower's FICO score and debt-to-income ratio.